In [1]:
# Install required packages
!pip install streamlit fasttext translate pyngrok --quiet
!wget https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin

# It appears that fasttext may not yet be updated for NumPy 2.0 so i shall downgrade the Numpy
!pip install numpy==1.24.3 --upgrade --force-reinstall

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 1.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.4 MB/s eta 0:00:00
--2025-05-15 20:32:38--  https://dl.fbaipublicfiles.com/fasttext/supervised-models/lid.176.bin
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 18.173.166.48, 18.173.166.74, 18.173.166.51, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|18.173.166.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 131266198 (125M) [application/octet-stream]
Saving to: ‘lid.176.bin’

lid.176.bin         100%[===================>] 1

In [1]:
# Import Libraries
import fasttext
from translate import Translator

# Load FastText Language Detection Model
model = fasttext.load_model('lid.176.bin')

# Define Detection and Translation Functions
def detect_language(text):
    label, _ = model.predict(text)
    return label[0].replace("__label__", "")

def translate_text(text, target_lang="ar"):
    translator = Translator(to_lang=target_lang)
    try:
        translated = translator.translate(text)
        return translated
    except Exception as e:
        return f"Translation failed: {e}"

In [ ]:
# Input from user
text_input = input("Enter a sentence to detect and translate: ")
target_language = input("Enter target language code (e.g., en, ar, fr, es): ")

# Detect and Translate
detected_lang = detect_language(text_input)
translated_text = translate_text(text_input, target_language)

# Output
print("\n--- Result ---")
print(f"Detected Language: {detected_lang}")
print(f"Translated to [{target_language}]: {translated_text}")

In [ ]:
import streamlit as st

# UI Layout
st.title("🌍 Language Identifier & Translator")

# Input Text
text_input = st.text_area("Enter text to detect and translate:")

# Language Options
language_options = {
    "English": "en",
    "Arabic": "ar",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Italian": "it",
    "Russian": "ru",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh"
}

target_language_name = st.selectbox("Choose target language:", list(language_options.keys()))
target_lang_code = language_options[target_language_name]

# Detect & Translate on Button Click
if st.button("Translate"):
    if text_input.strip():
        detected_lang = detect_language(text_input)
        translated = translate_text(text_input, target_lang_code)

        st.text_input("Detected Language:", detected_lang.upper())
        st.text_area("Translated Text:", translated)
    else:
        st.warning("Please enter some text to translate.")

In [7]:
# Use this name with
%%writefile app.py

# Import Libraries
import streamlit as st
import fasttext
from translate import Translator

# Load FastText Language Detection Model
model = fasttext.load_model('lid.176.bin')

# Define Detection and Translation Functions
def detect_language(text):
    label, _ = model.predict(text)
    print(label)
    return label[0].replace("__label__", "")

def translate_text(text, source_lang="en", target_lang="en"):
    translator = Translator(from_lang=source_lang,  to_lang=target_lang)
    try:
        translated = translator.translate(text)
        return translated
    except Exception as e:
        return f"Translation failed: {e}"

#####################
# UI Layout
st.title("🌐 Language Identifier & Translator")

# Input Text
text_input = st.text_area("Enter text to detect and translate:")

# Language Options
language_options = {
    "English": "en",
    "Arabic": "ar",
    "Spanish": "es",
    "French": "fr",
    "German": "de",
    "Italian": "it",
    "Russian": "ru",
    "Japanese": "ja",
    "Chinese (Simplified)": "zh"
}

target_language_name = st.selectbox("Choose target language:", list(language_options.keys()))
target_lang_code = language_options[target_language_name]

# Detect & Translate on Button Click
if st.button("Translate"):
    if text_input.strip():
        detected_lang = detect_language(text_input)
        translated = translate_text(text_input, detected_lang, target_lang_code)

        st.text_input("Detected Language:", detected_lang.upper())
        st.text_area("Translated Text:", translated)
    else:
        st.warning("Please enter some text to translate.")

Overwriting app.py


In [8]:
!ngrok config add-authtoken 2wxyTFi2pbVYCu0onO7gN90zmL3_4qwWg1yVyeAjj9pcTsEYk

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [9]:
from pyngrok import ngrok
import time
import threading
import os

# Kill previous tunnels if needed
ngrok.kill()

# Start streamlit in the background
def run():
    os.system("streamlit run app.py")

thread = threading.Thread(target=run)
thread.start()

# Wait for it to initialize
time.sleep(5)

# Set up the tunnel explicitly as http
public_url = ngrok.connect(8501, "http")
print("🌐 Your Streamlit app is live at:", public_url)


🌐 Your Streamlit app is live at: NgrokTunnel: "https://cafb-35-196-15-191.ngrok-free.app" -> "http://localhost:8501"
